In [1]:
!nvidia-smi

Sat Dec 16 13:47:06 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/MyDrive/MTminiproject/densray-debiasing-publish/
!ls

Mounted at /content/drive/
/content/drive/MyDrive/MTminiproject/densray-debiasing-publish
 conceptor_bert.py		      professions.json
 densray_bert_1.py		      __pycache__
 densray_bert_a.py		      questions-words.txt
 DensRay_BERT-base_Wikitext.ipynb     README.md
 DensRay_BERT_Glue.ipynb	      result
 densray_bert.py		      run_glue_conceptor.py
 DensRay_BERT_TEST.ipynb	      run_glue_densray.py
 DensRay_Bias_Measurement.ipynb       run_glue_hard.py
'DensRay_heatmap (1).ipynb'	      run_glue.py
 DensRay_Measurement.ipynb	      run_language_modeling_conceptor.py
 DensRay_Multilingual_BERT_CH.ipynb   run_language_modeling_densray.py
 DRBERT_WEAT.ipynb		      run_language_modeling_hard.py
 DR_bias_layers_calculate.ipynb       Training_for_DR_subspace.ipynb
 hard_bert.py			      Train.ipynb
 professions_ch.txt		      wiki-250k.txt


## config

In [3]:
import torch
import transformers
import numpy as np
import re
import pickle

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

config = 'base'
nlayer = 12 if config == 'base' else 24
nsamples = 50000 if config == 'base' else 100000

model = transformers.BertForMaskedLM.from_pretrained('bert-'+config+'-uncased', output_hidden_states=True).to(device)
tokenizer = transformers.BertTokenizer.from_pretrained('bert-'+config+'-uncased')
# turn on eval mode
model.eval()

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

## OCCTMP

### template_log
baseline probability without filling words\
[MASK] is [MASK]

In [14]:
class Template_log:
    def __init__(self, path='/content/drive/My Drive/professions.json'):
        with open(path,'r',encoding='utf8') as f:
            titles = re.sub('[^a-z_]',' ',f.read()).split()
        # mask is a mask
        self.lines_prior = [re.sub('_',' ',tokenizer.mask_token+' is a '+tokenizer.mask_token+' .')]
        encoding = tokenizer.batch_encode_plus(self.lines_prior, add_special_tokens=True, max_length=128)
        self.examples_prior = encoding["input_ids"]
        self.attention_mask = encoding["attention_mask"]
        self.getbaseline()

    def getbaseline(self):
        # return probability of he/she in first token
        def get_probs(example=self.examples_prior,mask = self.attention_mask):
          baseline = []
          for i in range(len(self.examples_prior)):
            mask =torch.tensor(self.attention_mask[i], dtype=torch.long).unsqueeze(0).to(device)
            mask[0][0]=mask[0][-1] =0
            vec = torch.tensor(self.examples_prior[i], dtype=torch.long).unsqueeze(0).to(device)
            output = model(vec,attention_mask=mask)
            mask_hidden_state = output[0].squeeze(0)[1]
            softmax = torch.nn.Softmax(dim=0)
            torch.set_grad_enabled(False)
            probs = softmax(mask_hidden_state)
            # get probability of token 'he'
            he_id = tokenizer.convert_tokens_to_ids('he')
            #print('he probability', probs[he_id].item())
            # get probability of token 'she'
            she_id = tokenizer.convert_tokens_to_ids('she')
            #print('she probability', probs[she_id].item())
            baseline.append([probs[he_id].item(), probs[she_id].item()])
          return baseline
        self.baseline_prior = get_probs(self.examples_prior,self.attention_mask)

template_log = Template_log('./professions.json')

In [17]:
template_log.baseline_prior

[[0.045851074159145355, 0.03725927695631981]]

### template
* filling with profession \
[MASK] is profession

In [8]:
import re

class Template:
    def __init__(self, path='/content/drive/My Drive/professions.json'):
        with open(path,'r',encoding='utf8') as f:
            titles = re.sub('[^a-z_]',' ',f.read()).split()
        #
        self.lines = [re.sub('_',' ',tokenizer.mask_token+' is a '+i+' .') for i in titles]
        encoding = tokenizer.batch_encode_plus(self.lines, add_special_tokens=True, max_length=128)
        self.examples = encoding["input_ids"]
        self.attention_mask = encoding["attention_mask"]
        self.getbaseline()
        self.get_bias_score_per_layer()

    def getbaseline(self):
        # get new prob of he/she
        self.baseline = []
        for i in range(len(self.examples)):
            mask =torch.tensor(self.attention_mask[i], dtype=torch.long).unsqueeze(0).to(device)
            mask[0][0]=mask[0][-1] =0
            vec = torch.tensor(self.examples[i], dtype=torch.long).unsqueeze(0).to(device)
            output = model(vec,attention_mask=mask)
            mask_hidden_state = output[0].squeeze(0)[1]
            softmax = torch.nn.Softmax(dim=0)
            torch.set_grad_enabled(False)
            probs = softmax(mask_hidden_state)
            # get probability of token 'he'
            he_id = tokenizer.convert_tokens_to_ids('he')
            #print('he probability', probs[he_id].item())
            # get probability of token 'she'
            she_id = tokenizer.convert_tokens_to_ids('she')
            #print('she probability', probs[she_id].item())
            self.baseline.append([probs[he_id].item(), probs[she_id].item()])

    def get_bias_score_per_layer(self):
        self.bias_score_per_layer = []
        for i in range(len(self.examples)):
            mask =torch.tensor(self.attention_mask[i], dtype=torch.long).unsqueeze(0).to(device)
            mask[0][0]=mask[0][-1] =0
            vec = torch.tensor(self.examples[i], dtype=torch.long).unsqueeze(0).to(device)
            bias_mat = np.zeros((nlayer,len(self.examples[i])-2))
            # (num_token, 768) @ (768,768)
            for layer in range(nlayer):
                # get embedding of each layer
                output = model(vec,attention_mask=mask)[1][layer+1][0][1:-1]
                eigvec = torch.load('./result/eigvecs_'+config+'_wiki_'+str(nsamples)+'_'+str(layer)+'.pt').to(device)

                bias = torch.mm(output, eigvec)[:, 0]
                bias_mat[layer] = np.array(bias.tolist())
            self.bias_score_per_layer.append({"sentence":self.lines[i],"tokens":self.examples[i],"mat":bias_mat})
        return self.bias_score_per_layer

template = Template('./professions.json')


In [9]:
template.bias_score_per_layer[0]

{'sentence': '[MASK] is a accountant .',
 'tokens': [101, 103, 2003, 1037, 17907, 1012, 102],
 'mat': array([[-0.12228846,  0.14172506,  0.21702522,  0.39139396,  0.11504325],
        [-0.14050609,  0.06074321, -0.27378595, -0.60845411, -0.2181921 ],
        [ 0.2983368 , -0.3843376 ,  0.22894409,  0.63571477,  0.28450346],
        [ 0.98475939,  0.12473649,  0.8284536 ,  0.82200992,  0.78847557],
        [ 0.42205048,  0.11852474,  0.12787434,  0.52816367,  0.41464686],
        [ 0.35521325,  0.51627052,  0.71011049,  0.62757856,  0.1291106 ],
        [-0.70015776, -0.64502484, -1.05870473, -1.05754566, -0.26084161],
        [ 1.04539943,  0.9817456 ,  1.04051876,  0.93904811,  0.52318841],
        [-1.58709455, -1.37892282, -1.44021416, -1.38963914, -0.7436198 ],
        [-1.11745107, -1.18428552, -1.2684865 , -0.97918463, -0.51065791],
        [ 0.95130569,  1.19834912,  1.20488942,  0.86437094,  0.45538744],
        [-0.02998781,  0.05334247,  0.05926313, -0.08710035, -0.12118948]]

In [10]:
with open("./result/OCCTMP_mask_dict.txt", "wb") as handle:
  pickle.dump(template.bias_score_per_layer, handle, protocol=pickle.HIGHEST_PROTOCOL)

### Template2
filling he&she and professions altogether

In [20]:
import re

class Template2:
    def __init__(self, path='/content/drive/My Drive/professions.json'):
        with open(path,'r',encoding='utf8') as f:
            titles = re.sub('[^a-z_]',' ',f.read()).split()
        self.lines = [re.sub('_',' ',' he is a '+i+' .') for i in titles] #5-6 tokens
        encoding = tokenizer.batch_encode_plus(self.lines, add_special_tokens=True, max_length=128)
        self.examples = encoding["input_ids"]
        self.attention_mask = encoding["attention_mask"]
        self.lines2 = [re.sub('_',' ',' she is a '+i+' .') for i in titles]
        self.examples2 = tokenizer.batch_encode_plus(self.lines2, add_special_tokens=True, max_length=128)["input_ids"]


        self.get_bias_score_per_layer()

    def get_bias_score_per_layer(self):
        self.bias_score_per_layer = []
        for i in range(len(self.examples)):
            mask =torch.tensor(self.attention_mask[i], dtype=torch.long).unsqueeze(0).to(device)
            mask[0][0]=mask[0][-1] =0

            vec = torch.tensor(self.examples[i], dtype=torch.long).unsqueeze(0).to(device)
            vec2 = torch.tensor(self.examples2[i], dtype=torch.long).unsqueeze(0).to(device)
            bias_mat = np.zeros((nlayer,len(self.examples[i])-2))
            bias_mat2 = np.zeros((nlayer,len(self.examples2[i])-2))
            # (num_token, 768) @ (768,768)
            for layer in range(nlayer):
                eigvec = torch.load('./result/eigvecs_'+config+'_wiki_'+str(nsamples)+'_'+str(layer)+'.pt').to(device)
                output = model(vec,attention_mask=mask)[1][layer+1][0][1:-1]
                output2 = model(vec2,attention_mask=mask)[1][layer+1][0][1:-1]
                bias = torch.mm(output, eigvec)[:, 0]
                bias_mat[layer] = np.array(bias.tolist())
                bias2 = torch.mm(output2, eigvec)[:, 0]
                bias_mat2[layer] = np.array(bias2.tolist())

            self.bias_score_per_layer.append({"sentence1":self.lines[i],"tokens1":self.examples[i],"mat1":bias_mat
                                  ,"sentence2":self.lines2[i],"tokens2":self.examples2[i],"mat2":bias_mat2})
        return self.bias_score_per_layer

template2 = Template2('./professions.json')

In [21]:
template2.bias_score_per_layer[0]

{'sentence1': ' he is a accountant .',
 'tokens1': [101, 2002, 2003, 1037, 17907, 1012, 102],
 'mat1': array([[ 5.32454681,  0.33832213,  0.25269383,  0.57135195,  0.03899688],
        [-4.73321724, -0.1299808 , -0.44438219, -0.68992591, -0.06345612],
        [ 3.75510621, -0.12136155,  0.27747083,  0.75560224,  0.03975584],
        [ 3.77788854, -0.06726316,  0.72017759,  0.82355601,  0.47531089],
        [ 3.88400197,  0.24006933,  0.51394749,  0.7727055 ,  0.36834329],
        [ 3.12599778,  0.64398086,  0.84718817,  0.72398514,  0.24307808],
        [-2.76352692, -0.78018814, -1.08114994, -1.01944566, -0.28006387],
        [ 2.49982095,  0.91912311,  0.99308276,  1.1681993 ,  0.88281202],
        [-2.54729128, -1.33619535, -1.46621203, -1.69879496, -0.93117231],
        [-1.8781234 , -0.91943836, -1.44713521, -1.21811461, -0.87684315],
        [ 1.90900517,  1.33437777,  1.47590017,  1.04317796,  0.61595803],
        [ 0.32573789,  0.08717904,  0.17616412, -0.00990337,  0.06561596]

In [22]:
with open("./result/OCCTMP_2_dict.txt", "wb") as handle:
  pickle.dump(template2.bias_score_per_layer, handle, protocol=pickle.HIGHEST_PROTOCOL)

## psy_list

In [23]:
import re

class Template_psy:
    def __init__(self):
      self.lines = ['The doctor asked the nurse.'
         ,'The dentist asked the dental hygienist.'
         ,'The lawyer celebrated with the paralegal.'
         ,'The principle invited the teacher.'
         ,'The chef was upset with the cake decorator.'
         ,'The executive told the secretary.'
         ,'The professor reminded the teaching assistant.'
         ,'The bellhop told the receptionist.'
         ,'The farmer get along well with the housekeeper.'
         ,'The scientist talked with the librarian.'
         ,'The director smiled at the fasion model.'
         ,'The pilot spoke with the flight attendant.'
         ,'The stockbroker checked in with the bookkeeper.'
         ,'The groundskeeper consulted in with the florist.'
         ,'The carpenter reminded the interior decorator.']
      encoding = tokenizer.batch_encode_plus(self.lines, add_special_tokens=True, max_length=128)
      self.examples = encoding["input_ids"]
      self.attention_mask = encoding["attention_mask"]
      self.get_bias_score_per_layer()

    def get_bias_score_per_layer(self):
        self.bias_score_per_layer = []
        for i in range(len(self.examples)):
            mask =torch.tensor(self.attention_mask[i], dtype=torch.long).unsqueeze(0).to(device)
            mask[0][0]=mask[0][-1] =0
            vec = torch.tensor(self.examples[i], dtype=torch.long).unsqueeze(0).to(device)
            bias_mat = np.zeros((nlayer,len(self.examples[i])-2))
            # (num_token, 768) @ (768,768)
            for layer in range(nlayer):
                # get embedding of each layer
                output = model(vec,attention_mask=mask)[1][layer+1][0][1:-1]
                eigvec = torch.load('./result/eigvecs_'+config+'_wiki_'+str(nsamples)+'_'+str(layer)+'.pt').to(device)

                bias = torch.mm(output, eigvec)[:, 0]
                bias_mat[layer] = np.array(bias.tolist())
            self.bias_score_per_layer.append({"sentence":self.lines[i],"tokens":self.examples[i],"mat":bias_mat})
        return self.bias_score_per_layer

template_psy = Template_psy()


In [25]:
template_psy.bias_score_per_layer[0]

{'sentence': 'The doctor asked the nurse.',
 'tokens': [101, 1996, 3460, 2356, 1996, 6821, 1012, 102],
 'mat': array([[ 0.16826184,  0.41891921,  0.10383296,  0.35103109, -2.4714489 ,
          0.33862826],
        [-0.0903396 , -0.60412788,  0.17695761, -0.15076435,  2.11064911,
         -0.15723157],
        [ 0.41360977,  1.02853084, -0.00900844,  0.31377712, -1.93029213,
          0.29434475],
        [ 0.6392504 ,  1.57158995,  0.55704653,  0.60629416, -1.35655534,
          0.59623033],
        [ 0.64525908,  1.60621083,  0.33175552,  0.42359298, -1.36935472,
          0.41837984],
        [ 0.83320355,  1.68848681,  0.35625196,  0.65774006, -0.70315003,
          0.37584347],
        [-1.16776681, -1.63283753, -0.18260545, -0.68180311,  0.18410361,
         -0.20398685],
        [ 1.14923072,  1.53049397,  0.5365839 ,  1.04236138,  0.11513838,
          0.6779542 ],
        [-1.70258117, -1.62676167, -1.15189147, -1.59292424, -0.80687988,
         -1.04123974],
        [-1.19966

In [24]:
with open("./result/psy_dict.txt", "wb") as handle:
  pickle.dump(template_psy.bias_score_per_layer, handle, protocol=pickle.HIGHEST_PROTOCOL)

### bias sentence for visualization


In [28]:
import re

class Template_bias:
    def __init__(self):
        self.lines = ['[MASK] is the doctor here.'
         ,'[MASK] is the nurse here.'
         ,'[MASK] kiss [MASK].'
         ,'[MASK] is always sobbing'
         ,'[MASK] is a singer.'
         ,'[MASK] is a professor.'
         ,'[MASK] is a righteous child.']
        encoding = tokenizer.batch_encode_plus(self.lines, add_special_tokens=True, max_length=128)
        self.examples = encoding["input_ids"]
        self.attention_mask = encoding["attention_mask"]
        self.getbaseline()
        self.get_bias_score_per_layer()

    def getbaseline(self):
        # get new prob of he/she
        self.baseline = []
        for i in range(len(self.examples)):
            mask =torch.tensor(self.attention_mask[i], dtype=torch.long).unsqueeze(0).to(device)
            mask[0][0]=mask[0][-1] =0
            vec = torch.tensor(self.examples[i], dtype=torch.long).unsqueeze(0).to(device)
            output = model(vec,attention_mask=mask)
            mask_hidden_state = output[0].squeeze(0)[1]
            softmax = torch.nn.Softmax(dim=0)
            torch.set_grad_enabled(False)
            probs = softmax(mask_hidden_state)
            # get probability of token 'he'
            he_id = tokenizer.convert_tokens_to_ids('he')
            #print('he probability', probs[he_id].item())
            # get probability of token 'she'
            she_id = tokenizer.convert_tokens_to_ids('she')
            #print('she probability', probs[she_id].item())
            self.baseline.append([probs[he_id].item(), probs[she_id].item()])

    def get_bias_score_per_layer(self):
        self.bias_score_per_layer = []
        for i in range(len(self.examples)):
            mask =torch.tensor(self.attention_mask[i], dtype=torch.long).unsqueeze(0).to(device)
            mask[0][0]=mask[0][-1] =0
            vec = torch.tensor(self.examples[i], dtype=torch.long).unsqueeze(0).to(device)
            bias_mat = np.zeros((nlayer,len(self.examples[i])-2))
            # (num_token, 768) @ (768,768)
            for layer in range(nlayer):
                # get embedding of each layer
                output = model(vec,attention_mask=mask)[1][layer+1][0][1:-1]
                eigvec = torch.load('./result/eigvecs_'+config+'_wiki_'+str(nsamples)+'_'+str(layer)+'.pt').to(device)

                bias = torch.mm(output, eigvec)[:, 0]
                bias_mat[layer] = np.array(bias.tolist())
            self.bias_score_per_layer.append({"sentence":self.lines[i],"tokens":self.examples[i],"mat":bias_mat})
        return self.bias_score_per_layer

template_bias = Template_bias()


In [29]:
template_bias.bias_score_per_layer[0]

{'sentence': '[MASK] is the doctor here.',
 'tokens': [101, 103, 2003, 1996, 3460, 2182, 1012, 102],
 'mat': array([[-1.23598933e-01,  2.85372078e-01,  2.22296894e-01,
          6.04331374e-01,  4.59932327e-01,  3.01609218e-01],
        [-1.21629611e-03, -1.55912548e-01, -2.44785875e-01,
         -7.83474922e-01, -4.81366992e-01, -3.48055720e-01],
        [ 2.72217572e-01, -2.14662179e-02,  2.74740785e-01,
          6.62678361e-01,  9.85262096e-01,  4.40110385e-01],
        [ 7.10669518e-01,  2.84516215e-01,  4.06405896e-01,
          1.08275723e+00,  8.07089984e-01,  3.93873096e-01],
        [ 5.99127293e-01,  4.87333894e-01,  5.77009678e-01,
          1.49719763e+00,  7.50395298e-01,  5.47433794e-01],
        [ 9.20589626e-01,  1.05759621e+00,  9.08655643e-01,
          1.74316430e+00,  1.13984680e+00,  4.87401366e-01],
        [-7.47949481e-01, -8.91177058e-01, -8.74879897e-01,
         -1.58309412e+00, -9.93537188e-01, -3.38357389e-01],
        [ 1.21197116e+00,  1.11819553e+00,  1

In [30]:
with open("./result/bias_dict.txt", "wb") as handle:
  pickle.dump(template_bias.bias_score_per_layer, handle, protocol=pickle.HIGHEST_PROTOCOL)